In [1]:
import os
import sys
import optuna
import shutil
import os.path as osp
import pytorch_lightning as pl

from pathlib import Path
from argparse import ArgumentParser, Namespace
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from datamodule import DataModule
from supervised import SupervisedLightningModule

import sys
sys.path.append('../')
from utils import run_cli, yaml_func


/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
default_config_path = './configs/config.yaml'
config = run_cli(config_path=default_config_path)

In [9]:
print(type(config['hparams']['k']))

<class 'float'>


In [12]:


def main(config_path) -> None:
    config = run_cli(config_path=config_path)
    seeds = config['seeds']
    
    kkkkk=[0.001,0.01,0.1,1]
    for i in range(len(kkkkk)):# step in k here
        config['hparams']['k']=kkkkk[i]
    
        for seed in seeds:
            if seed is not None:
                pl.seed_everything(seed)

            ckpt_callback = ModelCheckpoint(
                filename='{epoch}-{val_acc:.2f}'+str(config['hparams']['k']),#named with k
                **config['ckpt_callback'],
            )
            if 'callbacks' in config['trainer_params']:
                config['trainer_params']['callbacks'] = yaml_func(
                    config['trainer_params']['callbacks'])
            if config['trainer_params']['default_root_dir'] == "None":
                config['trainer_params']['default_root_dir'] = osp.dirname(__file__)
        
            model = SupervisedLightningModule(config)

            logger = TensorBoardLogger(
                save_dir=config['logger']['save_dir'],
                name=config['logger']['name']+f"-seed{seed}",
                version=config['logger']['version'],)
        
            dest_dir = os.path.join(config['logger']['save_dir'], config['logger']['name']+f"-seed{seed}", f"{config['logger']['version']}")

            trainer = pl.Trainer(**config['trainer_params'],
                            callbacks=[ckpt_callback],
                            logger=logger)
            imdm = DataModule(
                train_dir=config['dataset']['train_dir'],
                val_dir=config['dataset']['val_dir'],
            batch_size=config['hparams']['batch_size'])
            trainer.fit(model, datamodule=imdm)
            shutil.copy(config_path, f'{dest_dir}/config.yaml')


In [13]:
if __name__ == '__main__':
  default_config_path = './configs/config.yaml'
  main(default_config_path)

Global seed set to 4
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | InvertedResNet9  | 2.8 M 
1 | ce_loss | CrossEntropyLoss | 0     
---------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.307    Total estimated model params size (MB)


Validation sanity check:  50%|█████     | 1/2 [00:00<00:00,  1.86it/s]

/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Global seed set to 4


Epoch 0:  83%|████████▎ | 196/236 [01:21<00:16,  2.43it/s, loss=2.55, v_num=dam|]
Validating: 0it [00:00, ?it/s]
Epoch 1:  83%|████████▎ | 196/236 [01:23<00:16,  2.37it/s, loss=1.99, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  83%|████████▎ | 196/236 [01:25<00:17,  2.30it/s, loss=1.8, v_num=dam|]  
Validating: 0it [00:00, ?it/s]
Validating:  70%|███████   | 28/40 [00:05<00:02,  5.44it/s]


...

Epoch 92:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.167, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 93:  83%|████████▎ | 196/236 [02:10<00:26,  1.50it/s, loss=0.176, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Validating:  95%|█████████▌| 38/40 [00:08<00:00,  4.48it/s]


Epoch 94:  83%|████████▎ | 196/236 [02:10<00:26,  1.50it/s, loss=0.172, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 95:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.169, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 96:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.198, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 96:  94%|█████████▍| 222/236 [02:16<00:08,  1.63it/s, loss=0.198, v_num=dam|]


Epoch 97:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.169, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 98:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.163, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Epoch 99:  83%|████████▎ | 196/236 [02:10<00:26,  1.51it/s, loss=0.173, v_num=dam|] 
Validating: 0it [00:00, ?it/s]
Validating:  30%|███       | 12/40 [00:02<00:06,  4.51it/s]


Epoch 99: 100%|██████████| 236/236 [02:19<00:00,  1.69it/s, loss=0.173, v_num=dam|]


/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/pytorch_lightning/trainer/deprecated_api.py:32: LightningDeprecationWarning: `Trainer.train_loop` has been renamed to `Trainer.fit_loop` and will be removed in v1.6.
  rank_zero_deprecation(
